## Step-1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step-2: Download VisDrone Video dataset from Google Drive

In [ ]:
%%time
%%bash
if [ -d VisDroneVideo ]
then
  echo "data already exists"
else 
  mkdir -p VisDroneVideo
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-val.zip
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-test-dev.zip
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-train.zip
  # unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-test-challenge.zip
fi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 3.39 s, sys: 727 ms, total: 4.12 s
Wall time: 2min 46s


## Step-3: Convert to Visdrone-Video-YOLO format



In [ ]:
import os
from pathlib import Path
import shutil
import cv2
import csv
def _visdrone_video_to_yolov7_image_files(data_folder_dir, yolov7_output_folder_dir, new_image_size):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
    """
    # init paths/folders    
    input_ann_folder = str(Path(data_folder_dir) / "annotations")
    print(input_ann_folder)
    annotation_filepath_list = os.listdir(input_ann_folder)
    output_image_folder = str(Path(yolov7_output_folder_dir)/"images")
    Path(output_image_folder).mkdir(parents=True, exist_ok=True)
    image_size_dict = {}
    for annotation_filename in annotation_filepath_list:
      # print(annotation_filename)
      image_filepath_dir = os.path.splitext(str(Path(data_folder_dir) / "sequences"/annotation_filename))[0]
      image_filepath_list = os.listdir(image_filepath_dir)
      i = 0
      for image_name in image_filepath_list:
        yolov7_image_name = os.path.splitext(annotation_filename)[0] + "_" + image_name
        img_path = str(Path(image_filepath_dir)/image_name)
        img = cv2.imread(img_path)
        image_size_dict[annotation_filename] = img.shape
        resized_image = cv2.resize(img, new_image_size)
        resized_image_path = str(Path(output_image_folder)/yolov7_image_name)        
        try:
          cv2.imwrite(resized_image_path, resized_image)
        except:
          print("cannot write to: " + resized_image_path)
      
    return image_size_dict

In [ ]:
def _convert_visidrone_video_row_to_yolov7_row(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = int(box[2]) + int(box[4])/2.0
    y = int(box[3]) + int(box[5])/2.0
    w = int(box[4])
    h = int(box[5])    
    return (box[7], x*dw,y*dh,w*dw,h*dh)

In [ ]:
  
def _adjust_visidrone_video_row_for_image_resize(originalImageSize, newImageSize, box):
    oi_w = originalImageSize[1]
    oi_h = originalImageSize[0]
    ni_w = newImageSize[0]
    ni_h = newImageSize[1]
    # print(str(originalImageSize) + " ==> " + str(newImageSize))
    new_box = box.copy()
    new_box[2] = str(round(int(box[2])*(ni_w*1./oi_w)))
    new_box[3] = str(round(int(box[3])*(ni_h*1./oi_h)))
    new_box[4] = str(round(int(box[4])*(ni_w*1./oi_w)))
    new_box[5] = str(round(int(box[5])*(ni_h*1./oi_h)))
    # print("==========================")
    # print("new_box ==> " + str(new_box) + "; old_box ==> " + str(box))
    return new_box

In [ ]:
def _visdrone_video_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir, new_image_size, image_size_dict):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
        image_size_dict: dict
            contains the image_size info for various videos
    """
    # init paths/folders    
    input_ann_folder = str(Path(data_folder_dir) / "annotations")
    print(input_ann_folder)
    annotation_filepath_list = os.listdir(input_ann_folder)
    output_annotations_folder = str(Path(yolov7_output_folder_dir)/"labels")
    Path(output_annotations_folder).mkdir(parents=True, exist_ok=True)
    annotation_dict = {}
    for annotation_filename in annotation_filepath_list:
      image_size = image_size_dict[annotation_filename]
      print("image size for (h,w,c) " + annotation_filename + " ==> " + str(image_size))
      with open(Path(input_ann_folder)/annotation_filename) as file_obj:
        reader_obj = csv.reader(file_obj)
        i = 0
        for row in reader_obj:          
          yolov7_annotation_file_name = os.path.splitext(annotation_filename)[0] + "_" + str(row[0]).zfill(7) + ".txt"
          resized_row  =  _adjust_visidrone_video_row_for_image_resize(image_size, new_image_size, row) 
          yolov7_row = _convert_visidrone_video_row_to_yolov7_row(new_image_size, resized_row)
          # if i == 0:
            # print(annotation_filename + "==>" + str(row))          
            # print(yolov7_annotation_file_name + "==>" + str(yolov7_row))
          if yolov7_annotation_file_name not in annotation_dict:
            annotation_dict[yolov7_annotation_file_name] = []
          annotation_dict[yolov7_annotation_file_name].append(yolov7_row)
          i += 1
    for k, v in annotation_dict.items():
      with open(str(Path(output_annotations_folder)/k), "w", newline="") as f:
        for row in v:
          f.write(' '.join(str(item) for item in row)+'\n')

In [ ]:
#! rm -r /content/VisDroneVideo

In [ ]:
#! rm -r /content/VisDroneVideo/VisDrone2019-VID-val-yolov7

In [ ]:
def _convert_visdrone_data_to_yolov7_format(data_folder_dir, yolov7_output_folder_dir, _new_image_size):
  """
    Converts visdrone-VID annotations into yolov7 annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
  """
  image_size_dictionary = _visdrone_video_to_yolov7_image_files(data_folder_dir, yolov7_output_folder_dir, _new_image_size)
  print(image_size_dictionary)
  _visdrone_video_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir, _new_image_size, image_size_dictionary)

In [ ]:
! rm -r /content/VisDroneVideo/VisDrone2019-VID-YOLOv7

In [ ]:
def create_visdrone_video_data_in_yolov7_format(new_image_size = (960, 540)):
  
  _convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-train', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/train', new_image_size)
  _convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-val', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/val', new_image_size)
  _convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-test-dev', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/test-dev', new_image_size)
  ## Create data.yaml file
  try:
      with open('/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/data.yaml', 'w') as f:
          f.write('train: ../VisDroneVideo/VisDrone2019-VID-YOLOv7/train/images\n')
          f.writelines('val: ../VisDroneVideo/VisDrone2019-VID-YOLOv7/val/images\n')
          f.writelines('dev: ../VisDroneVideo/VisDrone2019-VID-YOLOv7/test-dev/images\n')
          f.writelines('\n')
          f.writelines('nc: 12\n')
          f.writelines("names: ['ignore', 'pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor', 'others']\n")
  except FileNotFoundError:
      print("The 'docs' directory does not exist")
image_size = (960, 540) # 960 px width x 540 px height
create_visdrone_video_data_in_yolov7_format(new_image_size=image_size)

In [ ]:
%%time
%%bash
cd /content/VisDroneVideo/VisDrone2019-VID-YOLOv7
zip -r 'VisDrone2019-VID-YOLOv7.zip' *

In [ ]:
!cp "/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/VisDrone2019-VID-YOLOv7.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/YOLOv7"

### Clone the GIT Repository

In [ ]:
!git clone https://github.com/dnautiyal/MLOps4-Capstone.git

In [ ]:
%%time
%%bash
if [ -d YOLOv7_VID_Annotated_Images ]
then
  echo "data already exists"
else 
  mkdir -p YOLOv7_VID_Annotated_Images
fi

In [ ]:
!rm -r /content/YOLOv7_VID_Annotated_Images/*

In [ ]:
!python /content/MLOps4-Capstone/src/yolo_data_utils/yolo_draw_annotation_box.py --label_folder './VisDroneVideo/VisDrone2019-VID-YOLOv7/test-dev/labels' --raw_images_folder './VisDroneVideo/VisDrone2019-VID-YOLOv7/test-dev/images' --save_images_folder './YOLOv7_VID_Annotated_Images/' 

In [ ]:
%%time
%%bash
zip -r 'VisDrone2019-VID-YOLOv7-Annotated_Images_Test.zip' '/content/YOLOv7_VID_Annotated_Images'

In [ ]:
!cp "/content/VisDrone2019-VID-YOLOv7-Annotated_Images_Test.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/YOLOv7"